In [1]:
import os
import pprint

# Define the target directory
target_directory = r"C:\Users\pablosal\Desktop\gbbai-agent-architecture-lab"  # change your directory here

# Check if the directory exists
if os.path.exists(target_directory):
    # Change the current working directory
    os.chdir(target_directory)
    print(f"Directory changed to {os.getcwd()}")
else:
    print(f"Directory {target_directory} does not exist.")

Directory changed to C:\Users\pablosal\Desktop\gbbai-agent-architecture-lab


### Understanding Semantic Kernel Components

Before diving into the implementation, let's understand the key components of Semantic Kernel.

#### The Kernel

The Kernel is the central orchestrator in Semantic Kernel. It manages:

- **Connecting to AI Models**: Interfaces with various AI models via connectors.
- **Registering and Invoking Plugins**: Manages the lifecycle and execution of plugins.
- **Managing Memory and Context**: Maintains state and context across interactions.
- **Interacting with the Planner**: Coordinates with the Planner to achieve specified goals.

#### Connectors

Connectors allow the Kernel to interface with various AI models and services. They define how the Kernel communicates with these models, whether they're:

- **OpenAI Models**: Such as GPT-3.5, GPT-4.
- **Azure OpenAI Services**: Leveraging Microsoft's cloud-based AI capabilities.
- **Local Models**: Using libraries like Hugging Face Transformers.

#### Plugins

Plugins are modular units that extend the Kernel's functionality. They consist of:

- **Prompt Functions**: Use natural language prompts to interact with AI models.
- **Native Functions**: Written in Python, performing deterministic tasks or interfacing with external services.

Plugins act as building blocks for complex workflows.

#### The Planner

The Planner uses AI to dynamically create a sequence of actions (a plan) to achieve a specified goal. It considers:

- **Available Plugins and Their Functions**: Understands the capabilities of each plugin.
- **Function Descriptions**: Uses metadata to understand what each function does.
- **Combining Functions**: Determines how functions can be combined to fulfill the goal.


### 🌟 Creating a Multi-Agent Architecture for Medical Research Documentation

In the realm of medical research, producing high-quality documentation is crucial. This guide aims to design a multi-agent system consisting of three specialized agents:

- **Medical Researcher**: Gathers and summarizes relevant medical information.
- **Clinical Evaluator**: Assesses the clinical relevance and accuracy of the information.
- **Medical Editor**: Refines the language and ensures adherence to medical writing standards.

Using Semantic Kernel's capabilities, we'll create plugins representing these agents and use a Planner to dynamically orchestrate their workflow based on a given goal.

By leveraging Semantic Kernel's capabilities, we can create a robust single-agent and sophisticated multi-agent system. Let's do it !

### Implementation

We will:

1. **Define Plugins**: Create plugins for the Medical Researcher, Clinical Evaluator, and Medical Editor.
2. **Configure the Kernel**: Set up the Kernel with the necessary connectors and plugins.
3. **Develop the Planner**: Implement the Planner to dynamically orchestrate the workflow.
4. **Execute the Plan**: Run the system to produce high-quality medical documentation.

In [2]:
#!pip install semantic-kernel --upgrade
import semantic_kernel as sk
print(f"We are using the following semantic_kernel library version: {sk.__version__}")

We are using the following semantic_kernel library version: 1.17.0


## 1. Define Plugins

We have defined the plugin for MedicalAgents located in the specified directory  `C:\Users\pablosal\Desktop\gbbai-agent-architecture-lab\src\plugins\plugins_store`

## 2. Configure The Kernel

In [3]:
import semantic_kernel as sk

# Initialize the kernel
kernel = sk.Kernel()

In [4]:
# add Azure OpenAI service connector to the kernel
import os
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

# Load environment variables from a .env file
from dotenv import load_dotenv
load_dotenv()

AZURE_OPENAI_KEY = os.getenv("AZURE_OPENAI_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_API_ENDPOINT")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
AZURE_OPENAI_EMBEDDING_DEPLOYMENT = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT")
AZURE_OPENAI_CHAT_DEPLOYMENT_ID = os.getenv("AZURE_AOAI_CHAT_MODEL_NAME_DEPLOYMENT_ID")

service_id = "openai-chat"
# Add Azure OpenAI chat completion
kernel.add_service(AzureChatCompletion(
    service_id=service_id,
    deployment_name=AZURE_OPENAI_CHAT_DEPLOYMENT_ID,
    api_key=AZURE_OPENAI_KEY,
    endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=AZURE_OPENAI_API_VERSION,
))

In [5]:
print("Registered services:", kernel.services)


Registered services: {'openai-chat': AzureChatCompletion(ai_model_id='gpt-4o-standard', service_id='openai-chat', client=<openai.lib.azure.AsyncAzureOpenAI object at 0x000001E2BD972860>, ai_model_type=<OpenAIModelTypes.CHAT: 'chat'>, prompt_tokens=0, completion_tokens=0, total_tokens=0)}


In [6]:
# Define the parent directory and plugin name
parent_directory = os.path.abspath(os.path.join("src", "plugins", "plugins_store"))
plugin_name = "medResearch"

# Add the plugin to the kernel
plugin = kernel.add_plugin(parent_directory=parent_directory, plugin_name=plugin_name)

print("Available functions in MedicalAgents plugin:")

print("Loaded plugin functions:", plugin.functions.keys())

print(plugin.functions['MedicalResearcher'].metadata)


Failed to create function from Python file: C:\Users\pablosal\Desktop\gbbai-agent-architecture-lab\src\plugins\plugins_store\medResearch\__init__.py


Available functions in MedicalAgents plugin:
Loaded plugin functions: dict_keys(['ClinicalEvaluator', 'MedicalEditor', 'MedicalResearcher'])
name='MedicalResearcher' plugin_name='medResearch' description='Gathers and summarizes the latest medical research on a given topic.' parameters=[KernelParameterMetadata(name='input', description='The topic for which to gather and summarize medical research.', default_value='', type_='', is_required=True, type_object=None, schema_data={'type': 'object', 'description': 'The topic for which to gather and summarize medical research.'}, include_in_function_choices=True)] is_prompt=True is_asynchronous=True return_parameter=KernelParameterMetadata(name='return', description='The completion result', default_value=None, type_='FunctionResult', is_required=True, type_object=None, schema_data=None, include_in_function_choices=True) additional_properties=None


In [7]:
from semantic_kernel.functions.kernel_arguments import KernelArguments

arguments = KernelArguments(input="What is the latest research on COVID-19?")
print("Arguments:", arguments)

result = await kernel.invoke(plugin.functions['MedicalResearcher'], arguments, service_id="openai-chat")
print("Result:", result)

Arguments: {'input': 'What is the latest research on COVID-19?'}
Result: ### Introduction

COVID-19, caused by the SARS-CoV-2 virus, has had a profound impact on global health since its emergence in late 2019. The pandemic has prompted an unprecedented amount of research aimed at understanding the virus, developing effective treatments, and creating vaccines to curb its spread. The significance of this research lies in its potential to save lives, reduce the burden on healthcare systems, and restore normalcy to societies worldwide.

### Key Findings

#### Study 1: Vaccine E


## 3. Develop the Planer

In [4]:
from semantic_kernel.connectors.ai.chat_completion_client_base import ChatCompletionClientBase
from semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.azure_chat_prompt_execution_settings import (
    AzureChatPromptExecutionSettings,
)
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior

In [8]:
from semantic_kernel.connectors.ai.chat_completion_client_base import ChatCompletionClientBase
from semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.azure_chat_prompt_execution_settings import (
    AzureChatPromptExecutionSettings,
)
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior

chat_completion : AzureChatCompletion = kernel.get_service(type=ChatCompletionClientBase)
execution_settings = AzureChatPromptExecutionSettings()
execution_settings.function_choice_behavior = FunctionChoiceBehavior.Auto() # the automatic planning loop https://learn.microsoft.com/en-us/semantic-kernel/concepts/planning?pivots=programming-language-python#the-automatic-planning-loop


## 4. Execute the Plan

In [9]:
from semantic_kernel.functions.kernel_arguments import KernelArguments
from semantic_kernel.contents.chat_history import ChatHistory
import logging
from semantic_kernel.utils.logging import setup_logging

# Create a history of the conversation
chat_history = ChatHistory()

# Add system message to the chat history to define the system's role
chat_history.add_system_message(
    "You are an AI-powered assistant specializing in producing high-quality medical research papers. "
    "Your goal is to assist users in creating comprehensive and accurate medical documents. "
    "You should generate relevant research, evaluate clinical data, and provide thorough evaluations to support the content."
)

# User message containing a detailed query
chat_history.add_user_message(
    "I would like to produce a high-quality medical document on the topic of 'The impact of AI on medical diagnostics.' "
    "Please provide detailed insights, key findings, and relevant research to support this topic. "
    "Additionally, include clinical evaluations and expert reviews to ensure the content is accurate and comprehensive."
)

# Set up logging
logging.basicConfig(
    format="[%(asctime)s - %(name)s:%(lineno)d - %(levelname)s] %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
setup_logging()
logging.getLogger("kernel").setLevel(logging.INFO)

# Get the response from the AI
result = (await chat_completion.get_chat_message_contents(
    chat_history=chat_history,
    settings=execution_settings,
    kernel=kernel,
    arguments=KernelArguments(input="The impact of AI on medical diagnostics"),
))[0]

# Print the result
print("AI Response:", result)

AI Response: ### The Impact of Artificial Intelligence on Medical Diagnostics

#### Introduction

The integration of Artificial Intelligence (AI) into medical diagnostics represents a transformative shift in the healthcare industry. AI's ability to analyze vast amounts of data with high precision and speed offers significant potential to enhance diagnostic accuracy, reduce human error, and improve patient outcomes. This topic is of paramount importance as it addresses the growing need for efficient and accurate diagnostic tools in an era of increasing healthcare demands and complexity.

#### Key Findings

##### AI in Radiology

A recent study published in the journal *Radiology* demonstrated the efficacy of AI algorithms in interpreting imaging studies. The research highlighted that AI systems could accurately detect abnormalities in chest X-rays, such as pneumonia, with sensitivity and specificity comparable to experienced radiologists. Moreover, the integration of AI tools in radiolo